In [25]:
import numpy as np
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.model_selection import train_test_split 
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier


def calculate_snr(real, imag, noise_power=1e-10):
    signal_power = np.sum(real**2 + imag**2, axis=1, keepdims=True)
    signal_power = np.maximum(signal_power, noise_power)  # Prevent very low or zero signal power
    snr = 10 * np.log10(signal_power / noise_power)
    return snr

# Function to calculate Willison Amplitude
def calculate_willison_amplitude(data, threshold=0.01):
    return np.sum(np.abs(np.diff(data, axis=1)) > threshold, axis=1, keepdims=True)

def calculate_decay_time_index(real, imag, p=0.5):
    h = np.sqrt(np.maximum(real**2 + imag**2, 1e-10))
    total_energy = np.sum(h**2, axis=1, keepdims=True)
    threshold = total_energy * p
    cumulative_energy = np.cumsum(h**2, axis=1)
    dti = np.argmax(cumulative_energy >= threshold, axis=1).reshape(-1, 1)  # Find the first index where cumulative energy exceeds threshold
    return dti

def total_energy(real, imag):
    return np.sum(real**2 + imag**2, axis=1, keepdims=True)



# Step 7: Sparse Coding Function
def find_sparse_coefficients(tSample, D, n_nonzero_coefs=15):
    omp = OrthogonalMatchingPursuit(n_nonzero_coefs=n_nonzero_coefs)
    omp.fit(D, tSample)
    return omp.coef_

# Step 8: Function to Calculate Residuals for Each Class
def calculate_residual(tSample, D, coefficients, class_indices):
    coef_class = np.zeros_like(coefficients)
    coef_class[class_indices] = coefficients[class_indices]  # Keep only coefficients for the specified class
    reconstructed_signal = D @ coef_class
    # Calculate squared residual
    residual = np.linalg.norm(tSample - reconstructed_signal) ** 2
    return residual

# Step 9: Classification Function
def classify_signal(tSample, D, trainLabel):
    coefficients = find_sparse_coefficients(tSample, D)
    residuals = []
    unique_classes = np.unique(trainLabel)
    for class_label in unique_classes:
        class_indices = np.where(trainLabel == class_label)[0]
        residual = calculate_residual(tSample, D, coefficients, class_indices)
        # print(f"Class: {class_label}, Residual: {residual}")
        residuals.append(residual)
    # print(residuals)
    # print(np.argmin(residuals))
    predicted_class = unique_classes[np.argmin(residuals)]
    return predicted_class


# Load the CIR dataset
measurement = np.load('../../dataset/meas_symm_3.npz', allow_pickle=False)
header, data = measurement['header'], measurement['data']
data_cir = data['cirs'][:8000]

# Define channels
alice_channel = 3  # Channel 3 is ALICE (legitimate)
eve_channel = 6  # Channel 6 is EVE (illegitimate)

# Step 1: Split the Original CIR Data into Train and Test Sets
train_cir, test_cir = train_test_split(data_cir, test_size=0.2, random_state=42)

# Step 2: Extract Features for Training Data
alice_train_CIRs = train_cir[:, alice_channel, :, :]
eve_train_CIRs = train_cir[:, eve_channel, :, :]




alice_train_real = alice_train_CIRs[:, :, 0]
alice_train_imag = alice_train_CIRs[:, :, 1]
alice_train_magnitude = np.abs(alice_train_real + 1j * alice_train_imag)
alice_train_snr = calculate_snr(alice_train_real, alice_train_imag)
alice_train_wa = calculate_willison_amplitude(alice_train_real)
alice_train_dti = calculate_decay_time_index(alice_train_real, alice_train_imag)
alice_train_energy = total_energy(alice_train_real, alice_train_imag)

alice_train_features = np.hstack((alice_train_real, alice_train_imag, alice_train_magnitude, alice_train_wa, alice_train_dti, alice_train_energy, alice_train_snr))

eve_train_real = eve_train_CIRs[:, :, 0]
eve_train_imag = eve_train_CIRs[:, :, 1]
eve_train_magnitude = np.abs(eve_train_real + 1j * eve_train_imag)
eve_train_snr = calculate_snr(eve_train_real, eve_train_imag)
eve_train_wa = calculate_willison_amplitude(eve_train_real)
eve_train_dti = calculate_decay_time_index(eve_train_real, eve_train_imag)
eve_train_energy = total_energy(eve_train_real, eve_train_imag)

eve_train_features = np.hstack((eve_train_real, eve_train_imag, eve_train_magnitude, eve_train_wa, eve_train_dti, eve_train_energy, eve_train_snr))

# Create labels for Alice and Eve for training
alice_train_labels = np.zeros(alice_train_features.shape[0])  # Label '0' for Alice.
eve_train_labels = np.ones(eve_train_features.shape[0])       # Label '1' for Eve.

# Combine data and labels for training
train_atoms = np.vstack((alice_train_features, eve_train_features))
train_labels = np.hstack((alice_train_labels, eve_train_labels))
print(f'atoms : {train_atoms.shape}')

# Step 3: Apply PCA for Dimensionality Reduction
scaler = StandardScaler()
train_atoms_normalized = scaler.fit_transform(train_atoms)
print(f'atoms normalized : {train_atoms_normalized.shape}')

pca = PCA(n_components=250)  # Reduce to 100 components (can be tuned)
train_atoms_pca = pca.fit_transform(train_atoms_normalized)
print(f'atoms normalized pca : {train_atoms_pca.shape}')

# Step 6: Form the Dictionary D from Training Data
D = train_atoms_pca.T
# print(f'D: {D.shape}')

# D = train_atoms.T



# Step 4: Extract Features for Test Data
alice_test_CIRs = test_cir[:, alice_channel, :, :]
eve_test_CIRs = test_cir[:, eve_channel, :, :]

alice_test_real = alice_test_CIRs[:, :, 0]
alice_test_imag = alice_test_CIRs[:, :, 1]
alice_test_magnitude = np.abs(alice_test_real + 1j * alice_test_imag)
alice_test_snr = calculate_snr(alice_test_real, alice_test_imag)
alice_test_wa = calculate_willison_amplitude(alice_test_real)
alice_test_dti = calculate_decay_time_index(alice_test_real, alice_test_imag)
alice_test_energy = total_energy(alice_test_real, alice_test_imag)

alice_test_features = np.hstack((alice_test_real, alice_test_imag, alice_test_magnitude, alice_test_wa, alice_test_dti, alice_test_energy, alice_test_snr))
# Extract Willison Amplitude for Test Data

eve_test_real = eve_test_CIRs[:, :, 0]
eve_test_imag = eve_test_CIRs[:, :, 1]
eve_test_magnitude = np.abs(eve_test_real + 1j * eve_test_imag)
eve_test_snr = calculate_snr(eve_test_real, eve_test_imag)
eve_test_wa = calculate_willison_amplitude(eve_test_real)
eve_test_dti = calculate_decay_time_index(eve_test_real, eve_test_imag)
eve_test_energy = total_energy(eve_test_real, eve_test_imag)
eve_test_features = np.hstack((eve_test_real, eve_test_imag, eve_test_magnitude, eve_test_wa, eve_test_dti, eve_test_energy, eve_test_snr))

# Create labels for Alice and Eve for testing
alice_test_labels = np.zeros(alice_test_features.shape[0])  # Label '0' for Alice.
eve_test_labels = np.ones(eve_test_features.shape[0])       # Label '1' for Eve.

# Combine data and labels for testing
test_atoms = np.vstack((alice_test_features, eve_test_features))
test_labels = np.hstack((alice_test_labels, eve_test_labels))

# Step 5: Apply PCA to Test Data
test_atoms_normalized = scaler.transform(test_atoms)
test_atoms_pca = pca.transform(test_atoms_normalized)


predictions = []

for testSample in test_atoms_pca:
    predicted_class = classify_signal(testSample, D, train_labels)
    predictions.append(predicted_class)
    # predictions.append(0)

predictions = np.array(predictions)
print(predictions.shape)

# Step 11: Calculate Accuracy
accuracy = np.mean(predictions == test_labels)
print(f"Classification Accuracy: {accuracy * 100:.2f}%")


atoms : (12800, 757)
atoms normalized : (12800, 757)
atoms normalized pca : (12800, 250)
(3200,)
Classification Accuracy: 77.19%


In [26]:
# Calculate confusion matrix
# print(f"\nTotal testing channel: {testData.shape}")

tn, fp, fn, tp = confusion_matrix(test_labels, predictions, labels=[0, 1]).ravel()

print(f"tp: {tp}")
print(f"tn: {tn}")
print(f"fp: {fp}")
print(f"fn: {fn}")

# # Missed Detection Rate (MDR)
MDR = fp / (fp + tn)

# # False Alarm Rate (FAR)
FAR = fn / (fn + tp)

# # Gamma calculation
gamma = (tp + fn) / (tn + fp)

# # Authentication Rate (AR)
AR = (tp + gamma * tn) / ((tp + fn) + gamma * (tn + fp))

print(f"MDR: {MDR}")
print(f"FAR: {FAR}")
print(f"AR: {AR}")

tp: 1504
tn: 966
fp: 634
fn: 96
MDR: 0.39625
FAR: 0.06
AR: 0.771875
